# Assignment 13

1. Gunakan kombinasi berikut untuk dataset wave:
  - Feature extractor HOG dengan image size 128 dan 300
  - Feature extractor LBP dengan image size 128 dan 300

  Masing-masing kombinasi dilakukan training dengan 3 algoritma:
    - LinearSVC
    - Logistic Regression
    - Random Forest
  Gunakan random_state=10 apabila dibutuhkan!

  Lakukan eksperimen terhadap hyperparameter sesuai kebutuhan dan tentukan komposisi mana yang dapat menghasilkan accuracy dan F1 score tertinggi! Berikan kesimpulan dan jelaskan!

2. Lanjutkan untuk dataset wave:
  - Ambilah secarik kertas dan buatlah 30 data tambahan untuk training dengan komposisi:
    - 15 healthy (dengan gambar wave yang rapih)
    - 15 parkinson (dengan gambar wave yang lebih ireguler)
  - Lakukan training dengan komposisi feature extractor, ukuran gambar, algoritma, serta hyperparameter terbaik yang didapat untuk nomor 1!

  Jelaskan, apakah terdapat perbedaan dalam hasil evaluasi training setelah ditambahkan data baru?

## Import Library

In [ ]:
import os
import pandas as pd
from google.colab.patches import cv2_imshow
from skimage import feature
from imutils import paths
import numpy as np
import cv2 as cv

## Fungsi Preprocessing

In [ ]:
def preprocess(image, image_size=128):
		image = cv.cvtColor(image, cv.COLOR_BGR2GRAY) # Ubah mehjadi grayscale
		image = cv.resize(image, (image_size, image_size)) # Resize gambar menjadi suatu ukuran (default = 128)

		image = cv.threshold(image, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1] # Melakukan thresholding dan mengambil hasil gambar thresholding

		return image

## Fungsi Feature Extraction

In [ ]:
# fungsi mengekstrak feature metode 'hog'
def quantify_image_hog(image): # Histogram of Oriented Gradient (hog) features
	features = feature.hog(image, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")

	return features

In [ ]:
# fungsi mengekstrak feature metode 'lbp'
def quantify_image_lbp(image): # Local Binary Pattern  (lbp) features
  features = feature.local_binary_pattern(image, 24, 8, method="uniform")

  (hist, _) = np.histogram(features.flatten(), bins=np.arange(0, 27), range=(0, 26))

  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)

  return hist

## Fungsi Split Dataset

In [ ]:
# fungsi memisahkan dataset 'data' dan 'label'
def load_split(path, image_size=200, extraction_method='hog'):
  image_paths = list(paths.list_images(path))
  data = []
  labels = []

  for image_path in image_paths:
    label = image_path.split(os.path.sep)[-2]

    image = cv.imread(image_path)
    image = preprocess(image, image_size=image_size)

    if extraction_method == 'hog':
      features = quantify_image_hog(image)
    elif extraction_method == 'lbp':
      features = quantify_image_lbp(image)

    data.append(features)
    labels.append(label)

  return (np.array(data), np.array(labels))

## Load Dataset

In [ ]:
# mengunggah dataset
dataset_dir = 'drive/My Drive/BARU/wave'
train_path = os.path.join(dataset_dir, 'training')
test_path = os.path.join(dataset_dir, 'testing')

## Melatih dan Evaluasi Model

In [ ]:
# Library Pelatihan dan Evaluasi Model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# fungsi image classification

def image_class(extraction, resize):
  # membagi feature dan label pada data training
  (feature_train, label_train) = load_split(train_path, image_size=resize, 
                                          extraction_method=extraction)

  # membagi feature dan label pada data testing
  (feature_test, label_test) = load_split(test_path, image_size=resize,
                                        extraction_method=extraction)
  
  # Label Encoder
  laben = LabelEncoder()
  label_train = laben.fit_transform(label_train)
  label_test = laben.fit_transform(label_test)

  # Model Linear SVC
  model_lsvc = LinearSVC(loss='squared_hinge', C=100)
  model_lsvc.fit(feature_train, label_train)
  lebel_pred_lsvc = model_lsvc.predict(feature_test)

  # Model Logistic Regression
  model_lr = LogisticRegression()
  model_lr.fit(feature_train, label_train)
  label_pred_lr = model_lr.predict(feature_test)

  # Model Random Forest Classifier
  model_rfc = RandomForestClassifier(n_estimators=1000, criterion='entropy', 
                                     max_depth=5, max_features='auto', 
                                     random_state=10)
  model_rfc.fit(feature_train, label_train)
  label_pred_rfc = model_rfc.predict(feature_test)

  nama_model = ['Linear SVC', 'Logistic Regression', 'Random Forest Classifier']
  model = [model_lsvc, model_lr, model_rfc]
  label_pred = [lebel_pred_lsvc, label_pred_lr, label_pred_rfc]

  # Evaluasi model
  for i, label in enumerate(label_pred):
    print('Nilai Akurasi data test {}'.format(nama_model[i]))
    print('- Accuracy \t: {:.2f}'.format(model[i].score(feature_test, label_test)))
    print('- F1 \t\t: {:.2f}'.format(f1_score(label_test, label, average='macro')))
    print('- Precission \t: {:.2f}'.format(precision_score(label_test, label, average='macro')))
    print('- Recall \t: {:.2f}'.format(recall_score(label_test, label, average='macro')))
    print('\n')

## Jawaban No. 1

### HOG 128

In [ ]:
# Feature extractor HOG dengan image size 128
image_class('hog', 128)

Nilai Akurasi data test Linear SVC
- Accuracy 	: 0.67
- F1 		: 0.67
- Precission 	: 0.67
- Recall 	: 0.67


Nilai Akurasi data test Logistic Regression
- Accuracy 	: 0.67
- F1 		: 0.67
- Precission 	: 0.67
- Recall 	: 0.67


Nilai Akurasi data test Random Forest Classifier
- Accuracy 	: 0.73
- F1 		: 0.73
- Precission 	: 0.74
- Recall 	: 0.73




### HOG 300

In [ ]:
# Feature extractor HOG dengan image size 300
image_class('hog', 300)

Nilai Akurasi data test Linear SVC
- Accuracy 	: 0.73
- F1 		: 0.73
- Precission 	: 0.74
- Recall 	: 0.73


Nilai Akurasi data test Logistic Regression
- Accuracy 	: 0.70
- F1 		: 0.70
- Precission 	: 0.70
- Recall 	: 0.70


Nilai Akurasi data test Random Forest Classifier
- Accuracy 	: 0.73
- F1 		: 0.73
- Precission 	: 0.73
- Recall 	: 0.73




### LBP 128

In [ ]:
# # Feature extractor LBP dengan image size 128
image_class('lbp', 128)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Nilai Akurasi data test Linear SVC
- Accuracy 	: 0.57
- F1 		: 0.55
- Precission 	: 0.57
- Recall 	: 0.57


Nilai Akurasi data test Logistic Regression
- Accuracy 	: 0.50
- F1 		: 0.50
- Precission 	: 0.50
- Recall 	: 0.50


Nilai Akurasi data test Random Forest Classifier
- Accuracy 	: 0.53
- F1 		: 0.53
- Precission 	: 0.53
- Recall 	: 0.53




### LBP 300

In [ ]:
# Feature extractor LBP dengan image size 300
image_class('lbp', 300)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Nilai Akurasi data test Linear SVC
- Accuracy 	: 0.57
- F1 		: 0.56
- Precission 	: 0.57
- Recall 	: 0.57


Nilai Akurasi data test Logistic Regression
- Accuracy 	: 0.53
- F1 		: 0.53
- Precission 	: 0.53
- Recall 	: 0.53


Nilai Akurasi data test Random Forest Classifier
- Accuracy 	: 0.70
- F1 		: 0.70
- Precission 	: 0.70
- Recall 	: 0.70




## Kesimpulan:
- Dari tunning parameter yang telah dilakukan maka dipreoleh lah akurasi yang terbaik yaitu model linear SVC dengan metode feature extractor HOG dan ukuran gambar 300 dengan nilai akurasi = 73, f1_score = 73, precision = 74, dan recall 73 
- Dapat dilihat bahwa pada metode feature extractor HOG akan menghasilkan akurasi yang lebih bagus jika ukuran size ditingkatkan karena dengan bertambah banyak feature2 yang diekstraksi dari HOG semakin mudah untuk mengklasifikasi gambar.
- begitu juga sebaliknya dengan metode extractor LBP, ukuran size dinaikkan akurasinya juga bertambah namun belum dapat sebaik akurasi extractor HOG karena feature2 pad HOG lebih baik dalam mengklasifikasi gambar

#Jawaban No. 2

In [ ]:
# mengunggah dataset yang sudah ditambah data baru
dataset_dir = 'drive/My Drive/assigment/wave'
train_path = os.path.join(dataset_dir, 'training')
test_path = os.path.join(dataset_dir, 'testing')

In [ ]:
# Algoritma (linear SVC), feature extractor (HOG) dan size (300) terbaik no. 1
# Feature extractor HOG dengan image size 300
image_class('hog', 300)

Nilai Akurasi data test Linear SVC
- Accuracy 	: 0.63
- F1 		: 0.63
- Precission 	: 0.63
- Recall 	: 0.63


Nilai Akurasi data test Logistic Regression
- Accuracy 	: 0.60
- F1 		: 0.60
- Precission 	: 0.60
- Recall 	: 0.60


Nilai Akurasi data test Random Forest Classifier
- Accuracy 	: 0.73
- F1 		: 0.73
- Precission 	: 0.74
- Recall 	: 0.73




In [ ]:
# Dibandingkan dengan ukuran size yang lebih kecil
# Feature extractor HOG dengan image size 100
image_class('hog', 100)

Nilai Akurasi data test Linear SVC
- Accuracy 	: 0.73
- F1 		: 0.73
- Precission 	: 0.75
- Recall 	: 0.73


Nilai Akurasi data test Logistic Regression
- Accuracy 	: 0.80
- F1 		: 0.80
- Precission 	: 0.81
- Recall 	: 0.80


Nilai Akurasi data test Random Forest Classifier
- Accuracy 	: 0.80
- F1 		: 0.80
- Precission 	: 0.80
- Recall 	: 0.80




In [ ]:
# lokasi data yng ditambah dan belum ditambah
dir_b = 'drive/My Drive/assigment/wave/training/healthy'
dir_a = 'drive/My Drive/BARU/wave/training/healthy'

In [ ]:
# jumlah data sesudah ditambah
len(os.listdir(dir_b))

51

In [ ]:
# jumlah data sebelum ditambah
len(os.listdir(dir_a))

36

## Kesimpulan
- Data yang saya tambah secara manual
- Dengan komposisi feature extractor, ukuran gambar, algoritma, serta hyperparameter terbaik yang didapat untuk nomor 1 (algoritma Linear SVC, extractor HOG dan size 300), akurasi pada no. 2 malah berkurang.
- Hal ini disebakan karena semakin banyak data yang ditambah ke data training sebelumnya semakin banyak pula komputer belajar sehingga memberikan pattern baru dan dengan pattern yang baru tersebut belum dapat mengklasifikasi data testing dengan lebih baik ddari sebelumnya.
- Namun saat ukuran size pada extractor HOG dikecilkan menjadi 100, hasil akurasinya menjadi lebih meningkat.
- Dapat disimpulkan bahwa saat dataset ditambah pada data training dengan jumlah feature yang banyak (HOG size 300) maka komputer akan semakin komplek mengklasifikasi gambar namun saat featurenya disedikitkan (HOG size 100) maka setiap pola feature pada gambar semaikn jelas dan mempermudah untuk mengklasifikasi gambar.
- Dengan bertambah data training belum tentu meningkatkan akurasi selain juga harus diiringi jumlah feature2 pada gambar yang saling berkolerasi sehingga jika menambah dataset maka ukuran sizenya juga harus diatur agar mmendapatkan pattern yang sesuai dan juga mengatur hyperparameter model algoritma. 